# First commit

In [1]:
print('hello')

hello


In [9]:
#! pip install -r kroger/python-kroger-client/requirements.txt

In [13]:
import os
os.environ['KROGER_CLIENT_ID'] = 'randomwalk-cae608773226c3fd9897f53c5776d9803270383690547060061'
os.environ['KROGER_CLIENT_SECRET'] = 'A3fcTjyE606zZBCSfH6AL4U0yk1oObOr2oxlV7XV'
os.environ['KROGER_REDIRECT_URI'] = 'http://localhost:8888'
os.environ['KROGER_USERNAME'] = 'test'
os.environ['KROGER_PASSWORD'] = 'nothing'

In [4]:
import os, sys
sys.path.append('kroger/python-kroger-client/')

In [14]:
from python_kroger_client.client import (
    KrogerCustomerClient,
    KrogerServiceClient,
)

from python_kroger_client.config import (
    customer_username,
    customer_password,
    encoded_client_token,
    redirect_uri,
)

In [15]:
encoded_client_token

'cmFuZG9td2Fsay1jYWU2MDg3NzMyMjZjM2ZkOTg5N2Y1M2M1Nzc2ZDk4MDMyNzAzODM2OTA1NDcwNjAwNjE6QTNmY1RqeUU2MDZ6WkJDU2ZINkFMNFUweWsxb09iT3Iyb3hsVjdYVg=='

In [30]:
# service_client = KrogerServiceClient(encoded_client_token=encoded_client_token)
# products = service_client.search_products(term="", limit=10, location_id='02600845')


In [16]:
#     def get_locations(self, zipcode, within_miles=10, limit=5, chain='Kroger'):
#         params = get_mapped_params(locals())
#         endpoint = '/locations'

#         results = self._make_get_request(endpoint, params=params)
#         data = results.get('data')
#         return [Location.from_json(location) for location in data]


client = KrogerServiceClient(encoded_client_token)
# locs = client.get_locations(zipcode = '80302')
# print(locs)

In [17]:
client.get_locations(zipcode = 80302)

TypeError: 'NoneType' object is not iterable

In [ ]:
class KrogerClient:
    def _make_get_request(self, endpoint, params=None):
        url = API_URL + endpoint
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {self.token}',
        }

        response = requests.get(url, headers=headers, params=params)
        return json.loads(response.text)

In [12]:
def add_to_items(items, product, quantity):
    """ Adds specified item to list of items with given quantity and returns items
    Arguments:
        items {array} -- An array of items to purchase
        product {Product} -- The product we want to buy
        quantity {int} -- How much of the product we want to buy
    """
    item_to_add = {
        "upc": product.upc,
        "quantity": quantity
    }
    items.append(item_to_add)
    return items


print("==================================================")
print("============== SERVICE CLIENT ====================")
print("==================================================")
service_client = KrogerServiceClient(encoded_client_token=encoded_client_token)
products = service_client.search_products(term="Taco", limit=10, location_id='02600845')
print()
print("PRODUCTS")
print("==================================================")
print()
for p in products: print(p)
print()
locations = service_client.get_locations(37206, within_miles=10, limit=10)
print("LOCATIONS")
print("==================================================")
print()
for l in locations: print(l)
print()

============== SERVICE CLIENT ====================


TypeError: 'NoneType' object is not iterable

In [18]:
def kroger_auth(client_id,redirect_uri):
    auth_params = {'scope': 'product.compact',
                   'response_type': 'code',
                   'client_id': client_id,
                   'redirect_uri': redirect_uri,
                   }

    html = requests.get("https://api.kroger.com/v1/connect/oauth2/authorize",
                        params = auth_params)

    # Print the link to the approval page
    return print(html.url)

def kroger_access(auth_code):
    token_url = 'https://api.kroger.com/v1/connect/oauth2/token'

    qd = {'grant_type': 'authorization_code',
          'code': auth_code,
          'redirect_uri': redirect_uri,
          'client_id': client_id,
          'client_secret': client_secret}

    response = requests.post(token_url, data=qd, timeout=60)
    response = response.json()

    if 'error' in response.keys():
        return print("Access Unsuccessfully Granted! Please try again!")
    else:
        access_token = response['access_token']
        refresh_token = response['refresh_token']
        return ["Access Successfully Granted!",access_token, refresh_token]
    
def kroger_refresh(refresh_token):
    token_url = 'https://api.kroger.com/v1/connect/oauth2/token'

    qd = {'grant_type': 'refresh_token',
          'refresh_token': refresh_token,
          'client_id': client_id,
          'client_secret': client_secret}

    response = requests.post(token_url, data=qd, timeout=60)
    response = response.json()

    if 'error' in response.keys():
        return print("Access Unsuccessfully Granted! Please try again!")
    else:
        access_token = response['access_token']
        refresh_token = response['refresh_token']
        return ["Access Successfully Granted!",access_token, refresh_token]
    
def pull_product_data(category,resultlimit): 
    global access_token, refresh_token
    
    # get raw data
    url = 'https://api.kroger.com/v1/products?filter.term='+category+'&filter.locationId=01400441&filter.limit='+str(resultlimit)
    headers = {"content-type": "application/json; charset=UTF-8",'Authorization':'Bearer {}'.format(access_token)}
    try:
        response = requests.get(url, headers=headers).json()

        # refresh access token if it has expired 
        if 'error' in response.keys():
            refresh = kroger_refresh(refresh_token)
            access_token = refresh[1]
            refresh_token = refresh[2]
            headers = {"content-type": "application/json; charset=UTF-8",'Authorization':'Bearer {}'.format(access_token)}
            response = requests.get(url, headers=headers).json()
    except:
        response = {'data': []}
        
    df = json_normalize(response,record_path ='data')

    # get selling details
    items = pd.DataFrame()
    for i in range(0,df.shape[0]):
        temp = df['items'][i][0]
        for info in temp.keys():
            items.loc[i,'description'] = df.loc[i,'description']
            if type(temp[info]) == dict:
                for subinfo in temp[info].keys():
                    text = str(info+'_'+subinfo)
                    items.loc[i,text] = temp[info][subinfo]
            else:
                items.loc[i,info] = temp[info]

    return items, access_token, refresh_token




In [20]:
import requests

In [23]:
client_id = "randomwalk-cae608773226c3fd9897f53c5776d9803270383690547060061"
client_secret = 'A3fcTjyE606zZBCSfH6AL4U0yk1oObOr2oxlV7XV'
redirect_uri = 'http://localhost:8888'

# Request authentication URL. After running this function, click on the link generated to be taken to your 
# redirect page for authentication code.
kroger_auth(client_id,redirect_uri)

https://api.kroger.com/v1/connect/oauth2/authorize?scope=product.compact&response_type=code&client_id=randomwalk-cae608773226c3fd9897f53c5776d9803270383690547060061&redirect_uri=http%3A%2F%2Flocalhost%3A8888
